In [73]:
#IMPORT LIBRARIES
from sklearn.neural_network import MLPRegressor
import numpy as np
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.datasets import make_blobs, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import hvplot.pandas
import holoviews as hv

%matplotlib inline

In [4]:
#READ CSV FILES
GMA_data = pd.read_csv("data/GMA_data.csv")
Houston_data = pd.read_csv("data/Houston_data.csv")

In [5]:
#Display GMA_data
display(GMA_data.head())

,Geo Market Area,ASP 12/2021 - 12/2022,ASP 12/2022 - 12/2023,% Change,Transactions 12/2021 - 12/2022,Transactions 12/2022 - 12/2023,% Change.1,MOI 11/2022,MOI 11/2023,% Change.2,Avg DOM 12/2021 - 12/2022,Avg DOM 12/2022 - 12/2023,% Change.3
0,1960/Cypress,"$305,022","$319,320",4.70%,290,205.0,-29.30%,1.5,2.6,72.10%,15.0,35.7,138.00%
1,1960/Cypress Creek North,"$321,283","$331,624",3.20%,411,245.0,-40.40%,2.2,4.6,107.50%,27.6,39.2,42.00%
2,1960/Cypress Creek South,"$265,879","$258,609",-2.70%,786,613.0,-22.00%,2.2,2.9,28.20%,23.3,40.2,72.50%
3,Aldine Area,"$237,692","$237,042",-0.30%,880,755.0,-14.20%,2.5,2.9,14.60%,23.2,38.1,64.20%
4,Alief,"$260,950","$262,756",0.70%,523,443.0,-15.30%,2.5,3.2,25.50%,21.8,31.6,45.00%


In [86]:
#Read Historic_data.csv
Historic_data = pd.read_csv("data/Historic_pricing.csv")

df = Historic_data
#Display Historic_data
display(Historic_data.head())

,Unnamed: 0,Unnamed: 1,Closed,Average,Median,Price/,New,Active,Pending,Unnamed: 9,Close to
0,Year,Month,Listings*,Price,Price,Sqft,Listings,Listings,Listings,DOM,Orig. List Price
1,2021,Jan,"4,815","$321,238","$255,000",$132,"7,202","9,088","6,097",43,96.70%
2,2021,Feb,"4,671","$348,928","$270,000",$139,"6,148","7,781","5,721",42,97.20%
3,2021,Mar,"7,327","$363,397","$285,000",$143,"8,831","7,184","7,889",39,97.80%
4,2021,Apr,"7,459","$372,590","$290,500",$148,"10,164","7,725","8,195",33,98.80%


In [87]:
# Define features set
X = Historic_data.copy()
X.drop("Close to", axis=1, inplace=True)
X.head()

,Unnamed: 0,Unnamed: 1,Closed,Average,Median,Price/,New,Active,Pending,Unnamed: 9
0,Year,Month,Listings*,Price,Price,Sqft,Listings,Listings,Listings,DOM
1,2021,Jan,"4,815","$321,238","$255,000",$132,"7,202","9,088","6,097",43
2,2021,Feb,"4,671","$348,928","$270,000",$139,"6,148","7,781","5,721",42
3,2021,Mar,"7,327","$363,397","$285,000",$143,"8,831","7,184","7,889",39
4,2021,Apr,"7,459","$372,590","$290,500",$148,"10,164","7,725","8,195",33


In [92]:
# Print d-types
print(Historic_data.dtypes)

Unnamed: 0    object
Unnamed: 1    object
Closed        object
Average       object
Median        object
Price/        object
New           object
Active        object
Pending       object
Unnamed: 9    object
Close to      object
dtype: object


In [93]:
# Define target vector
y = Historic_data["Unnamed: 1"].values.reshape(-1, 1)
y[:5]

array([['Month'],
       ['Jan'],
       ['Feb'],
       ['Mar'],
       ['Apr']], dtype=object)

In [94]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)


In [91]:
# Create the StandardScaler instance
scaler = StandardScaler()



In [95]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)



ValueError: could not convert string to float: 'Year'

In [9]:
#Read Historic_data.csv
Historic_data = pd.read_csv("data/Historic_pricing.csv")

#Display Houston_data
display(Houston_data.head())
display(Houston_data.tail(5))

,Unnamed: 0,Closed,% Change,Unnamed: 3,% Change.1,Average,% Change.2,Median,% Change.3
0,Property Type,Listings*,Year Ago,Dollar Volume,Year Ago,Price,Year Ago,Price,Year Ago
1,All (New and Existing),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Single Family,"7,983",-4%,"$3,353,852,000",-1%,"$420,124",3%,"$339,000",0%
3,Townhouse/Condominium,556,-19%,"$145,065,478",-17%,"$260,909",3%,"$225,000",3%
4,Multifamily,65,-22%,"$31,514,889",-27%,"$484,844",-6%,"$390,000",-5%


,Unnamed: 0,Closed,% Change,Unnamed: 3,% Change.1,Average,% Change.2,Median,% Change.3
27,High Rise,2,-75%,"$2,799,000",-62%,"$1,399,500",51%,"$1,399,500",217%
28,Rentals - Single Family,330,4%,"$767,587",3%,"$2,326",-1%,"$2,300",0%
29,Rentals - TH/Condo,26,86%,"$57,961",66%,"$2,229",-11%,"$2,288",-7%
30,Rentals - Multifamily,36,16%,"$60,755",-11%,"$1,688",-23%,"$1,600",7%
31,Rentals - High Rise,2,-71%,"$2,575",-87%,"$1,288",-55%,"$1,288",NaN


In [10]:
#Drop Columns
df = Houston_data.drop("Closed",axis=1)